In [1]:
import os
import pandas as pd
import numpy as np
def unir_archivos_csv_carpeta(carpeta):
    archivos_csv = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.csv')]
    dfs = []
    
    for archivo in archivos_csv:
        ruta_archivo = os.path.join(carpeta, archivo)
        df = pd.read_csv(ruta_archivo)
        dfs.append(df)
    
    dataframe_final = pd.concat(dfs, ignore_index=True)
    return dataframe_final

def dumificar_columna(dataframe, columna):
    dummies = pd.get_dummies(dataframe[columna], prefix=columna, drop_first=True)
    dataframe = pd.concat([dataframe, dummies], axis=1)
    dataframe.drop(columna, axis=1, inplace=True)
    return dataframe

def generar_consecutivos(df, columna_orden, columna_consecutivos):
    # Ordenar el DataFrame según la columna de orden
    df_ordenado = df.sort_values(by=columna_orden)
    
    # Generar valores consecutivos en una nueva columna
    df_ordenado[columna_consecutivos] = range(1, len(df_ordenado) + 1)
    
    return df_ordenado

In [2]:
carpeta_archivos = 'C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/igac/REGISTRO_2'

dataframe_final = unir_archivos_csv_carpeta(carpeta_archivos)


C:\Users\ABI\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3214: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\Users\ABI\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3214: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [46]:
archivo_csv = 'https://drive.google.com/uc?export=download&id=1a9IfZDuiqbhh2iefH8BMarHL8IXWpayF'
Municipios = pd.read_csv(archivo_csv, sep=';', encoding='latin-1')
Municipios=Municipios['Id_Municipio_Dane']
Municipios=Municipios.to_frame()


In [4]:
# Area Promedio por lote
Area_mean_lote = dataframe_final.groupby('MUNICIPIO')['AREA_TERRENO_1'].mean()
Area_mean_lote=Area_mean_lote.to_frame()
Area_mean_lote = Area_mean_lote.rename(columns={'AREA_TERRENO_1':'Lotes_Mean_Area'})


In [5]:
# Numero de lotes
Numero_lotes = dataframe_final.groupby('MUNICIPIO')['AREA_TERRENO_1'].count()
Numero_lotes=Numero_lotes.to_frame()
Numero_lotes = Numero_lotes.rename(columns={'AREA_TERRENO_1':'Lotes_Cantidad'})


In [6]:
# Area promedio costruida (solo lotes con construcción)

df_filtrado = dataframe_final[dataframe_final['AREA_CONSTRUIDA_1'] > 0]
Area_contruida = df_filtrado.groupby('MUNICIPIO')['AREA_CONSTRUIDA_1'].mean()
Area_contruida=Area_contruida.to_frame()  
Area_contruida = Area_contruida.rename(columns={'AREA_CONSTRUIDA_1':'Lotes_Area_Mean_Construida'})
#Area_contruida


In [7]:
dataframe_final.columns

Index(['DEPARTAMENTO', 'MUNICIPIO', 'NUMERO_PREDIAL', 'ZONA_FISICA_1',
       'ZONA_ECONOMICA_1', 'AREA_TERRENO_1', 'ZONA_FISICA_2',
       'ZONA_ECONOMICA_2', 'AREA_TERRENO_2', 'HABITACIONES_1', 'BANOS_1',
       'LOCALES_1', 'PISOS_1', 'TIPIFICACION_1', 'USO_1', 'PUNTAJE_1',
       'AREA_CONSTRUIDA_1', 'HABITACIONES_2', 'BANOS_2', 'LOCALES_2',
       'PISOS_2', 'TIPIFICACION_2', 'USO_2', 'PUNTAJE_2', 'AREA_CONSTRUIDA_2',
       'HABITACIONES_3', 'BANOS_3', 'LOCALES_3', 'PISOS_3', 'TIPIFICACION_3',
       'USO_3', 'PUNTAJE_3', 'AREA_CONSTRUIDA_3', 'NUMERO_PREDIAL_ANTERIOR'],
      dtype='object')

In [8]:
# Numero lotes con construcciónes grandes, medianas, pequeñas,muy pequeños
categorias = pd.qcut(df_filtrado['AREA_CONSTRUIDA_1'], 4, labels=False)
categorias=categorias.to_frame()
categorias = categorias.rename(columns={'AREA_CONSTRUIDA_1':'Lotes_Cat_Area_Construida'})
dataframe_concat = pd.concat([df_filtrado['MUNICIPIO'], categorias], axis=1)
nueva_dataframe = pd.pivot_table(dataframe_concat, index=dataframe_concat.MUNICIPIO, columns='Lotes_Cat_Area_Construida', aggfunc='size', fill_value=0)
nueva_dataframe.columns = ['Lotes_Cat_Area_Construida_' + str(col) for col in nueva_dataframe.columns]
Cat_Area_Construida=nueva_dataframe
#nueva_dataframe = nueva_dataframe.rename(columns={0:'Lotes_Cat_Area_Construida_0'})


In [9]:
# Numero lotes de areas grandes, medianas, pequeñas,muy pequeños
categorias = pd.qcut(dataframe_final['AREA_TERRENO_1'], 4, labels=False)
categorias=categorias.to_frame()
categorias = categorias.rename(columns={'AREA_TERRENO_1':'Lotes_Cat_Area'})
dataframe_concat = pd.concat([dataframe_final['MUNICIPIO'], categorias], axis=1)
nueva_dataframe = pd.pivot_table(dataframe_concat, index=dataframe_concat.MUNICIPIO, columns='Lotes_Cat_Area', aggfunc='size', fill_value=0)
nueva_dataframe.columns = ['Lotes_Cat_Area_' + str(col) for col in nueva_dataframe.columns]
Cat_Area=nueva_dataframe
#nueva_dataframe = nueva_dataframe.rename(columns={0:'Lotes_Cat_Area_Construida_0'})



In [10]:
# en lotes con info, promedio de numero de habitaciones

df_filtrado = dataframe_final[dataframe_final['HABITACIONES_1'] > 0]
Habitaciones = df_filtrado.groupby('MUNICIPIO')['HABITACIONES_1'].mean()
Habitaciones=Habitaciones.to_frame()  
Habitaciones = Habitaciones.rename(columns={'HABITACIONES_1':'Lotes_Habitacion_promedio'})


In [11]:
# en lotes con info, promedio de numero de baños
df_filtrado = dataframe_final[dataframe_final['BANOS_1'] > 0]
Baños = df_filtrado.groupby('MUNICIPIO')['BANOS_1'].mean()
Baños=Baños.to_frame()  
Baños = Baños.rename(columns={'BANOS_1':'Lotes_Baños_promedio'})


In [12]:
# en lotes con info, promedio de numero de locales

df_filtrado = dataframe_final[dataframe_final['LOCALES_1'] > 0]
Locales = df_filtrado.groupby('MUNICIPIO')['LOCALES_1'].mean()
Locales = Locales.to_frame()  
Locales = Locales.rename(columns={'LOCALES_1':'Lotes_Numero_Locales_promedio'})

In [13]:
# cantidad de lotes por tipo de uso

dataframe_concat=dataframe_final[['MUNICIPIO', 'USO_1']]
nueva_dataframe = pd.pivot_table(dataframe_concat, index=dataframe_concat.MUNICIPIO, columns='USO_1', aggfunc='size', fill_value=0)
nueva_dataframe.columns = ['Lotes_Cat_Tipo_uso_' + str(col) for col in nueva_dataframe.columns]
Tipo_Uso=nueva_dataframe



In [14]:
# cantidad de lotes por tipificación

dataframe_concat=dataframe_final[['MUNICIPIO', 'TIPIFICACION_1']]
nueva_dataframe = pd.pivot_table(dataframe_concat, index=dataframe_concat.MUNICIPIO, columns='TIPIFICACION_1', aggfunc='size', fill_value=0)
nueva_dataframe.columns = ['Lotes_Cat_Tipificacion_' + str(col) for col in nueva_dataframe.columns]
Tipificacion=nueva_dataframe


In [15]:
# porcentaje lotes por categoria area cosntruida

# Calcular la suma por fila
suma_filas = Cat_Area_Construida.sum(axis=1)

# Calcular los porcentajes de cada valor en las columnas
porcentajes_cat_area_cosntruida = Cat_Area_Construida.div(suma_filas, axis=0) * 100


In [16]:
# porcentaje lotes por categoria area cosntruida

# Calcular la suma por fila
suma_filas = Cat_Area.sum(axis=1)

# Calcular los porcentajes de cada valor en las columnas
porcentajes_cat_area = Cat_Area.div(suma_filas, axis=0) * 100

In [17]:
# Numero de habitaciones como categoria

df=pd.DataFrame(dataframe_final['HABITACIONES_1'])
df_consecutivos = generar_consecutivos(df, 'HABITACIONES_1', 'HABITACIONES_1_1')
categorias=pd.qcut(df_consecutivos['HABITACIONES_1_1'], 10, labels=False).sort_index()
categorias=categorias.to_frame()
categorias = categorias.rename(columns={'HABITACIONES_1_1':'Lotes_Cat_Habitacion'})
dataframe_concat = pd.concat([dataframe_final['MUNICIPIO'], categorias], axis=1)
nueva_dataframe = pd.pivot_table(dataframe_concat, index=dataframe_concat.MUNICIPIO, columns='Lotes_Cat_Habitacion', aggfunc='size', fill_value=0)
nueva_dataframe.columns = ['Lotes_Cat_Habitacion_' + str(col) for col in nueva_dataframe.columns]
Cat_Lotes_Habitacion=nueva_dataframe


In [18]:
# Baños como categoria

df=pd.DataFrame(dataframe_final['BANOS_1'])
df_consecutivos = generar_consecutivos(df, 'BANOS_1', 'BANOS_1_1')
categorias=pd.qcut(df_consecutivos['BANOS_1_1'], 10, labels=False).sort_index()
categorias=categorias.to_frame()
categorias = categorias.rename(columns={'BANOS_1_1':'Lotes_Cat_Banos'})
dataframe_concat = pd.concat([dataframe_final['MUNICIPIO'], categorias], axis=1)
nueva_dataframe = pd.pivot_table(dataframe_concat, index=dataframe_concat.MUNICIPIO, columns='Lotes_Cat_Banos', aggfunc='size', fill_value=0)
nueva_dataframe.columns = ['Lotes_Cat_Banos_' + str(col) for col in nueva_dataframe.columns]
Cat_Lotes_Banos=nueva_dataframe


In [36]:
# Numero de locales como categoria

df=pd.DataFrame(dataframe_final['LOCALES_1'])
df_consecutivos = generar_consecutivos(df, 'LOCALES_1', 'LOCALES_1_1')
categorias=pd.qcut(df_consecutivos['LOCALES_1_1'], 4, labels=False).sort_index()
categorias=categorias.to_frame()
categorias = categorias.rename(columns={'LOCALES_1_1':'Lotes_Cat_Locales'})
dataframe_concat = pd.concat([dataframe_final['MUNICIPIO'], categorias], axis=1)
nueva_dataframe = pd.pivot_table(dataframe_concat, index=dataframe_concat.MUNICIPIO, columns='Lotes_Cat_Locales', aggfunc='size', fill_value=0)
nueva_dataframe.columns = ['Lotes_Cat_Locales_' + str(col) for col in nueva_dataframe.columns]
Cat_Lotes_Locales=nueva_dataframe


In [20]:
# porcentaje lotes por categoria de habitacion

# Calcular la suma por fila
suma_filas = Cat_Lotes_Banos.sum(axis=1)

# Calcular los porcentajes de cada valor en las columnas
porcentajes_Cat_Lotes_Habitacion = Cat_Lotes_Banos.div(suma_filas, axis=0) * 100

In [21]:
# porcentaje lotes por categoria de hbaitacion baños

# Calcular la suma por fila
suma_filas = Cat_Lotes_Banos.sum(axis=1)

# Calcular los porcentajes de cada valor en las columnas
porcentajes_Cat_Lotes_Banos = Cat_Lotes_Banos.div(suma_filas, axis=0) * 100


In [38]:
# porcentaje lotes por categoria de locales

# Calcular la suma por fila
suma_filas = Cat_Lotes_Locales.sum(axis=1)

# Calcular los porcentajes de cada valor en las columnas
porcentajes_Cat_Lotes_Locales = Cat_Lotes_Locales.div(suma_filas, axis=0) * 100


In [62]:
df_unido=Municipios.merge(Area_mean_lote, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Numero_lotes, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Area_contruida, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Cat_Area_Construida, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Cat_Area, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Habitaciones, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Baños, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Locales, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Tipo_Uso, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Tipificacion, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(porcentajes_cat_area_cosntruida, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(porcentajes_cat_area, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Cat_Lotes_Habitacion, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Cat_Lotes_Banos, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Cat_Lotes_Locales, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(porcentajes_Cat_Lotes_Habitacion, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(porcentajes_Cat_Lotes_Banos, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(porcentajes_Cat_Lotes_Locales, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')




In [26]:
shape = df_unido.shape
  
# Printing Number of columns
print('Number of columns :', shape[1])
#df_unido

Number of columns : 174


In [64]:
df_unido.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Lotes.csv', index=False)


In [67]:
import json

# Ruta del archivo .sd
ruta_archivo = 'C:/Users/ABI/Downloads/BASE_CATASTRAL_NACIONAL_11_2022.sd'

# Leer el archivo .sd
with open(ruta_archivo, 'r', encoding='latin-1') as archivo:
    contenido = archivo.read()

# Convertir el contenido del archivo en un objeto JSON
datos = json.loads(contenido)

# Ahora puedes acceder a los datos del archivo como un diccionario
print(datos)

JSONDecodeError: Extra data: line 1 column 2 (char 1)

In [70]:
import pandas as pd
# Leer el archivo CSV como DataFrame
df = pd.read_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/lotes/TOTAL_NACIONAL_REG2.csv', sep=',',encoding='latin-1' )

C:\Users\ABI\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,1,3,6,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [72]:
df['DEPARTAMENTO'].unique()

array([8, 13, 15, nan, 17, 18, 19, 20, 23, 25, 27, 41, 44, 47, 50, 52, 54,
       63, 66, 68.0, 70, 73, 76, 81, 85, 86, 88, 91, 94, 95.0, 97, 99,
       '99', '.'], dtype=object)